# Data cleaning step

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

### Loading data

In [3]:
# Drop first 5 rows of metadata
raw = pd.read_csv('outage.csv', header=None).iloc[5:]

# Extract and clean column names from the first actual row (index 5 in the original file)
cols = raw.iloc[0, 1:].tolist()
# Drop the first column (contains "variables") and the header row itself
raw = raw.iloc[1:, 1:].copy()

# Assign cleaned column names
raw.columns = cols

# Reset index so we are working with the correct row numbers.
raw.reset_index(drop=True, inplace=True)

# Finally, drop variable column 
raw = raw.iloc[1:, :]
raw.head(5)

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
3,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
4,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
5,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


### Renaming columns

In [4]:
new_cols = [
    'obs', 'year', 'month', 'state', 'postal_code', 'nerc_region',
    'climate_region', 'anomaly_level', 'climate_cat',
    'start_date', 'start_time', 'restore_date',
    'restore_time', 'cause_cat', 'cause_detail',
    'hurricane_names', 'duration', 'demand_loss_mw',
    'customers_affected', 'res_price', 'com_price', 'ind_price',
    'total_price', 'res_sales', 'com_sales', 'ind_sales', 'total_sales',
    'res_pct', 'com_pct', 'ind_pct', 'res_customers',
    'com_customers', 'ind_customers', 'total_customers', 'res_cust_pct',
    'com_cust_pct', 'ind_cust_pct', 'pc_realgsp_state', 'pc_realgsp_usa',
    'pc_realgsp_rel', 'pc_realgsp_change', 'util_realgsp', 'total_realgsp',
    'util_contri', 'pi_util_of_usa', 'population', 'pop_pct_urban',
    'pop_pct_uc', 'popden_urban', 'popden_uc', 'popden_rural',
    'area_pct_urban', 'area_pct_uc', 'pct_land', 'pct_water_tot',
    'pct_water_inland'
]

raw.columns = new_cols
raw.head(2)


,obs,year,month,state,postal_code,nerc_region,climate_region,anomaly_level,climate_cat,start_date,...,pop_pct_urban,pop_pct_uc,popden_urban,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


## Brainstorming problem statments
(⚠️ means still need more data, ✅ means workable with existing data )

**Currently, we are considering three problem statement to explore:**

1. ⚠️ Assessment of Infrastructure Resilience (Based on whether or not `cause_detail` has enough information.)
Analyze how different regions' infrastructure characteristics (e.g., overhead vs. underground lines, maintenance investments) correlate with outage frequency and duration. This can inform infrastructure improvement strategies.
2. ✅ Temporal Trends and Climate Change Correlation (doable with existing data, but kinda boring) 
Examine how the frequency and causes of outages have evolved over time and assess potential correlations with climate change indicators. This can provide insights into how changing climate patterns impact power reliability.
3. ⚠️ Policy and Emergency Response Evaluation (We don’t have explicit timestamps or markers indicating policy changes or emergency response dates.)
Evaluate the effectiveness of policies and emergency responses by analyzing outage data before and after the implementation of specific measures. This can guide future policy development and emergency planning.

**Ideas that are just "ok":**

⭕️ 1. Predictive Modeling of Outage Risks <u> ***(but it’s kinda boring, i bet people have done it already; not creative)*** </u>

Utilize machine learning techniques to predict the likelihood of major power outages based on factors such as weather patterns, infrastructure characteristics, and economic indicators. This can aid in proactive maintenance and resource allocation.

⭕️ 2. Socioeconomic Impact Analysis <u> ***(has been done numerous times already !!!!!!! Check out examples, two of them already exist. I know you like it, Im sry!)*** </u>

Investigate the relationship between socioeconomic factors (e.g., income levels, urbanization) and the frequency or duration of power outages. This can highlight areas where outages disproportionately affect vulnerable populations.



### Exploring idea 1
Check out whether `cause_detail` gives us any good information about the infrustructure. 

In [5]:
raw['cause_detail'].unique()

array([nan, 'vandalism', 'heavy wind', 'thunderstorm', 'winter storm',
       'tornadoes', 'sabotage', 'hailstorm', 'uncontrolled loss',
       'winter', 'wind storm', 'computer hardware', 'public appeal',
       'storm', ' Coal', ' Natural Gas', 'hurricanes', 'wind/rain',
       'snow/ice storm', 'snow/ice ', 'transmission interruption',
       'flooding', 'transformer outage', 'generator trip',
       'relaying malfunction', 'transmission trip', 'lightning',
       'switching', 'shed load', 'line fault', 'breaker trip', 'wildfire',
       ' Hydro', 'majorsystem interruption', 'voltage reduction',
       'transmission', 'Coal', 'substation', 'heatwave',
       'distribution interruption', 'wind', 'suspicious activity',
       'feeder shutdown', '100 MW loadshed', 'plant trip', 'fog', 'Hydro',
       'earthquake', 'HVSubstation interruption', 'cables', 'Petroleum',
       'thunderstorm; islanding', 'failure'], dtype=object)

- `cause_detail` does contain some infrastructure-related failure types, like:
**transformer outage, generator trip, relaying malfunction, breaker trip, line fault, substation, transmission interruption, distribution interruption, cables, HVSubstation interruption, plant trip**, etc.
- They will allow us to indirectly infer infrastructure issues, but there are no explicit infrastructure metadata: We don’t have direct info on overhead vs underground lines, age of equipment, maintenance budgets, or investments.

- External datasets to consider: 
    - **(I FW THIS ONE HEAVY) [EIA Reports on Utility Investments](https://www.eia.gov/todayinenergy/detail.php?id=48136)** 
        - Provides financial and operational data related to maintenance and upgrades.
        - Can help explain or correlate investment levels with outage frequency/duration.
        - Good source for explaining patterns seen in outage data.
    - [Mapping the Depths: Underground Power Distribution (arXiv study, paper only)](https://arxiv.org/abs/2402.06668)
        - Unique dataset that quantifies underground vs overhead lines by utility.
        - Can provide a strong predictor variable about infrastructure type (underground = more resilient).

### Exploring EIA reports on utility investment 
-> <u> **[Annual Electric Power Industry Report, Form EIA-861 detailed data files](https://www.eia.gov/electricity/data/eia861/)**</u> <br>
-> <u> **[A Guide to EIA Electric Power Data](https://www.eia.gov/electricity/data/guide/pdf/guide.pdf)** page 9/18  </u> 
> **Retail Sales by Electric Utilities and Power Marketers (Form EIA-861, Annual Electric Power Industry Report)**
>
> Data Collected by Form EIA-861  
> The Form EIA-861, Annual Electric Power Industry Report collects annual data from a census of all utilities that sell electricity to end-use customers in the 50 states, the District of Columbia, Puerto Rico, American Samoa, the American Virgin Islands, Guam, and the Northern Mariana Islands. These surveys collect information on sales to ultimate customers by utilities and power marketers, energy efficiency programs, distributed generating capacity, and related data elements.  
>
> The data collected include several items:  
> - **Service territory by state and county**  
> - **Sales revenue to ultimate customers**  
> - **Revenue and customer count**  
> - **Source and disposition of electricity**  
> - **Advanced metering**  
> - **Demand response and energy efficiency programs**  
> - **Dynamic pricing**  
> - **Capacity and other information related to net metering**  
> - **Non-net metered distributed generating units**  
> - **Distribution system characteristics and reliability**


Out of the above, the good columns to look for are ` Distribution System Characteristics and Reliability`, ` Service Territory by State and County`, `Advanced Metering Infrastructure (AMI)`, `Revenue and Customer Count by Utility` 
-  `Distribution System Characteristics and Reliability`: Directly speaks to infrastructure resilience. 
-  `Service Territory by State and County`: Needed for merging 
- `Advanced Metering Infrastructure (AMI)`: AMI often correlates with modernization efforts and may reflect better outage response times. Can assess: Compare outage duration/frequency in regions with vs. without AMI.
- `Demand Response and Energy Efficiency Programs`: May suggest proactive infrastructure investment or mitigation strategies. Can assess: "Do regions with stronger demand response programs show fewer or shorter outages?"
- `Revenue and Customer Count by Utility`: Can assess possible correlations between revenue and investment in resilience.

In [6]:
raw.head(2)

,obs,year,month,state,postal_code,nerc_region,climate_region,anomaly_level,climate_cat,start_date,...,pop_pct_urban,pop_pct_uc,popden_urban,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


# Basic visualizations 

In [7]:
fig = px.box(
    raw,
    x='state',
    y='anomaly_level',
    color='state',  # Optional: colors each box by state
    title='Anomaly Levels by State',
)

fig.update_layout(
    xaxis_title='State',
    yaxis_title='Anomaly Level',
    title_font=dict(size=30),
    xaxis_tickangle=90,
    font=dict(size=12),
    height=400,
    width=800,
    showlegend=False  # You can turn this on if coloring by state is meaningful
)

fig.show()


In [8]:
# Create a combined label for clarity
data = raw.copy()
data['state_region'] = data['state'] + ' (' + data['nerc_region'] + ')'

fig = px.scatter(
    data,
    x='state_region',
    y='duration',
    color='year',
    title='Outage Duration by State and NERC Region (Colored by Year)',
    labels={'State_Region': 'State (NERC Region)', 'Outage Duration': 'Outage Duration (hours)'},
    hover_data=['state', 'nerc_region', 'year', 'duration'],
)

fig.update_layout(
    xaxis_tickangle=60,
    xaxis_title='State and NERC Region',
    yaxis_title='Outage Duration',
    height=700,
    width=1400,
    font=dict(size=14)
)

fig.show()


In [9]:
# Assuming 'year' is an integer
# Convert 'year' to numeric (will turn bad values into NaN)
data['year'] = pd.to_numeric(data['year'], errors='coerce')
data['year_group'] = (data['year'] - data['year'] % 3).astype(str) + '–' + (data['year'] - data['year'] % 3 + 2).astype(str)
fig = px.box(
    data,
    x='state',
    y='duration',
    color='nerc_region',
    facet_col='year_group',
    title='Outage Duration by State (Grouped in 3-Year Periods)',
    labels={'duration': 'Outage Duration (hours)', 'year_group': 'Year Group'}
)

fig.update_layout(
    height=500,
    width=3000,
    font=dict(size=14),
    xaxis_tickangle=60
)

fig.show()



## Decide whether the dataset is in the scope of this project

Professor said as long as we are still mainly using the dataset given, we will be fine. 
Let's inspect our own dataset and think about a fallback plan. 


Current idea we want to use: **Assessment of Infrastructure Resilience**
Analyze how different regions' infrastructure characteristics (e.g., overhead vs. underground lines, maintenance investments) correlate with outage frequency and duration. This can inform infrastructure improvement strategies.


If we do use the outside dataset, the steps we will MAINLY involve the dataset will be: 
- Introduction: We will have to explain the external dataset, but it's ok. 
- Data cleaning and EDA: Can do on the main dataset. All data visualizations can be done for original dataframe. We can add stuff to it. 
- Assessment of missingess can be completely done on our own dataset. 
- Hypothesis testing: Can be done on the dataset itself. 
- Predictive problem: the problem is, Step 5 - 8 will be all depending on the external dataset. I don't know if that is allowed. 

# Final decision on problem statement 

**Assessment of Infrastructure Resilience (Based on whether or not `cause_detail` has enough information.)
Analyze how different regions' infrastructure characteristics (e.g., overhead vs. underground lines, maintenance investments) correlate with outage frequency and duration. This can inform infrastructure improvement strategies.**



## Step 2: Data Cleaning and Exploratory Data Analysis

# TODO: 1. Inspect and clean missing data from original dataframe 
1. Inspect how much missing data there is in each column (percentage) (rename the dataframe to outage, so that there's destintion between raw dataset (used for missingness later) and real dataset we will be using)
2. Decide on how to fill in missingness (drop? mean imputation? probability imputation?)
3. Feature engineer however you think is fit. 

In [10]:
dropped = [
    'obs', 'start_date', 'start_time', 'restore_date', 'restore_time',
    'res_price', 'com_price', 'ind_price',
    'res_sales', 'com_sales', 'ind_sales',
    'res_pct', 'com_pct', 'ind_pct',
    'res_customers', 'com_customers', 'ind_customers',
    'res_cust_pct', 'com_cust_pct', 'ind_cust_pct',
    'pct_land', 'pct_water_tot', 'pct_water_inland',
    'hurricane_names', 'postal_code', 'pc_realgsp_usa', 
    'pc_realgsp_rel', 'pc_realgsp_change', 'util_realgsp',
    'total_realgsp', 'util_contri', 'nerc_region',
    'demand_loss_mw', 'customers_affected',
    'total_price', 'total_sales', 'total_customers',   
]

In [11]:
raw_drop = raw.drop(columns = dropped)
raw_drop = raw_drop.fillna(np.nan)

In [12]:
(raw_drop.isna().sum(axis = 0) / raw_drop.shape[0]).sort_values(ascending= False)

cause_detail        0.307040
duration            0.037810
popden_uc           0.006519
popden_rural        0.006519
anomaly_level       0.005867
month               0.005867
climate_cat         0.005867
climate_region      0.003911
year                0.000000
state               0.000000
cause_cat           0.000000
pi_util_of_usa      0.000000
pc_realgsp_state    0.000000
population          0.000000
pop_pct_urban       0.000000
popden_urban        0.000000
pop_pct_uc          0.000000
area_pct_urban      0.000000
area_pct_uc         0.000000
dtype: float64

### Columns with notably large amounts of missing data:
demand_loss_mw
customers_affected
cause_detail
hurricane_names



# TODO 2: Process external data source 
1. Using the link on this website, download all zip files from 2000 - 2016 (Do NOT use the reformatted files!)
https://www.eia.gov/electricity/data/eia861/

2. Get file 2. Convert the file into CSV, Get the three total columns, keep columns of : Year, State, Thousands dollars, megawatthours, count for Total. Group each year's dataframe by state, sum for each column. 

3. For now, just have 17 dataframes.

Total revenue is very high, might want to regularize it?

In [ ]:
external = pd.DataFrame(columns = ['state', 'total_revenue', 'megawatthours', 'count', 'year'])

for i in range(17):
    if i < 10:
        num = '0' + str(i)
    else:
        num = str(i)
    state_col = 'Unnamed: 3' if i == 0 else 'Unnamed: 6'
    df = pd.read_csv(f'20{num}.csv')
    counts = df.columns[-1]
    megawatts = df.columns[-2]
    revenue = df.columns[-3]
    df = df.iloc[2:][[state_col, revenue, megawatts, counts]].rename(columns = {state_col: 'state', revenue: 'total_revenue', 
                                                                                            megawatts: 'megawatthours', counts: 'count'})
    df['total_revenue'] = df['total_revenue'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)*1000
    df['megawatthours'] = df['megawatthours'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)
    df['count'] = df['count'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)
    df = df.groupby('state').sum().reset_index()
    df['year'] = '20' + num
    external = pd.concat([external, df], axis = 0)

external['year'] = external['year'].astype(int)

C:\Users\monso\AppData\Local\Temp\ipykernel_17876\1914324587.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [14]:
raw

,obs,year,month,state,postal_code,nerc_region,climate_region,anomaly_level,climate_cat,start_date,...,pop_pct_urban,pop_pct_uc,popden_urban,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
3,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
4,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
5,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,"Tuesday, December 6, 2011",...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1531,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1532,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743
1533,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743


In [20]:
external['year_state'] = external['year'].astype(str) + external['state'].astype(str)
external

,state,total_revenue,megawatthours,count,year,year_state
0,AK,5.352460e+08,5309970.0,273530.0,2000,2000AK
1,AL,4.687257e+09,83524220.0,2262753.0,2000,2000AL
2,AR,2.399365e+09,41611188.0,1362838.0,2000,2000AR
3,AZ,4.431208e+09,61130045.0,2193562.0,2000,2000AZ
4,CA,2.289222e+10,264464209.0,12742304.0,2000,2000CA
...,...,...,...,...,...,...
46,VT,7.978707e+08,5516450.0,368070.0,2016,2016VT
47,WA,6.827187e+09,90970890.0,3382680.0,2016,2016WA
48,WI,7.443497e+09,69736338.0,3017146.0,2016,2016WI
49,WV,2.879482e+09,32076146.0,1014790.0,2016,2016WV


In [25]:
state_duration = raw.groupby(['year', 'postal_code'])['duration'].sum().reset_index().rename(columns = {'postal_code': 'state'})
state_duration['year_state'] = state_duration['year'].astype(str) + state_duration['state'].astype(str)
final_df = external.merge(state_duration, on = 'year_state', how = 'left')[['total_revenue', 'megawatthours', 'count', 'year_state', 'duration']].fillna(0)
final_df['year'] = final_df['year_state'].str[:4]
final_df['state'] = final_df['year_state'].str[4:]
final_df = final_df.drop(columns = 'year_state')
final_df.to_csv('raw_external_combined.csv')